In [ ]:
metadata = {}
data = {'1': '1'}
metadata | data | {'11': '11'}

In [ ]:
from datatrove.executor.local import LocalPipelineExecutor
from datatrove.pipeline.dedup import MinhashDedupSignature
from datatrove.pipeline.dedup.minhash import (
    MinhashConfig,
    MinhashDedupBuckets,
    MinhashDedupCluster,
    MinhashDedupFilter,
)
from datatrove.pipeline.readers import HuggingFaceDatasetReader
from datatrove.pipeline.writers.jsonl import JsonlWriter
from datatrove.pipeline.writers.huggingface import HuggingFaceDatasetWriter
from datatrove.pipeline.tokens import TokensCounter
from datatrove.utils.hashing import HashConfig

from eeve.data.utils import _reader_adapter_with_column_info, _writer_adapter_with_column_restore

In [ ]:
# INPUT_READER = HuggingFaceDatasetReader('alexantonov/chuvash_russian_parallel', text_key='ru', shuffle_files=False, dataset_options={'split':'train'})
INPUT_READER = HuggingFaceDatasetReader('alexantonov/chuvash_russian_parallel', text_key='ru', shuffle_files=False, dataset_options={'split':'train'}, adapter=_reader_adapter_with_column_info, doc_progress=True)

In [ ]:
# gen = INPUT_READER.run()
# doc = next(iter(gen))
# doc.metadata['chv']

In [ ]:
# doc.metadata

In [ ]:
# import dataclasses

# data = {key: val for key, val in dataclasses.asdict(doc).items() if val}

In [ ]:
# data |= data.pop("metadata")

In [ ]:
num_buckets = 40
hashes_per_bucket = 3
n_grams = 4
precision = 64
language = "rus_Cyrl"

total_tasks = 6
workers = 12

BASE_MINHASH_PATH = "/mnt/d/vscode_projects/eeve/eeve/workdir/minhash_tests"
minhash_base_path = f"{BASE_MINHASH_PATH}/minhash_parallel"
base_logging_dir = f"{BASE_MINHASH_PATH}/minhash_logs_parallel"
local_working_dir = f'{BASE_MINHASH_PATH}/minhash_local_parallel'

upload_path = 'whatisslove11/dedup_chv_parallel_overall_v123455644'

In [ ]:
minhash_config = MinhashConfig(
    hash_config=HashConfig(
        precision=precision,
        hash_fc="sha1"
    ),
    n_grams=n_grams,
    num_buckets=num_buckets,
    hashes_per_bucket=hashes_per_bucket,
)

stage1 = LocalPipelineExecutor(
    pipeline=[
        INPUT_READER,
        MinhashDedupSignature(
            output_folder=f"{minhash_base_path}/signatures",
            config=minhash_config,
            language=language
        ),
    ],
    logging_dir=f'{base_logging_dir}/signatures',
    tasks=total_tasks,
    workers=workers
)

stage2 = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupBuckets(
            input_folder=f"{minhash_base_path}/signatures",
            output_folder=f"{minhash_base_path}/buckets",
            config=minhash_config,
        ),
    ],
    depends=stage1,
    logging_dir=f'{base_logging_dir}/buckets',
    tasks=minhash_config.num_buckets,
    workers=workers
)

stage3 = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupCluster(
            input_folder=f"{minhash_base_path}/buckets",
            output_folder=f"{minhash_base_path}/remove_ids",
            config=minhash_config,
        ),
    ],
    depends=stage2,
    logging_dir=f'{base_logging_dir}/clusters',
    tasks=1,
    workers=workers
)

stage4 = LocalPipelineExecutor(
    pipeline=[
        INPUT_READER,
        TokensCounter(),  
        MinhashDedupFilter(
            input_folder=f"{minhash_base_path}/remove_ids",
            exclusion_writer=JsonlWriter(f"{minhash_base_path}/removed"),
        ),
        HuggingFaceDatasetWriter(
            dataset=upload_path,
            private=True,
            local_working_dir=local_working_dir, # необяз арг
            output_filename="data/${rank}.parquet",
            cleanup=True, # нужно ли удалять с локальной папки датасет потом
            adapter=_writer_adapter_with_column_restore
        ),
    ],
    depends=stage3,
    logging_dir=f'{base_logging_dir}/clusters',
    tasks=minhash_config.num_buckets,
    workers=workers
)

stage4.run()

In [ ]:
import pandas as pd
jsonObj = pd.read_json(path_or_buf='/mnt/d/vscode_projects/eeve/notebooks/minhash_parallel/removed/00001.jsonl.gz', lines=True)

In [ ]:
jsonObj['text'][13]

In [ ]:
from datasets import load_dataset

In [ ]:
dd = load_dataset('whatisslove11/dedup_chv_parallel_overall_v1234567') # upload_path

In [ ]:
xfd = dd['train']['ru']

In [ ]:
len(xfd)

In [ ]:
from tqdm.auto import tqdm


def find_substring_in_list(list_of_strings, substring):
    for string_item in tqdm(list_of_strings):
        if substring in string_item:
            return True, string_item
    return False, ""

In [ ]:
find_substring_in_list(dd['train']['ru'], '13011000112115200036992')

In [ ]:
import struct
import glob

def read_remove_file(filename):
    """Читает файл .remove и возвращает список чисел"""
    try:
        with open(filename, 'rb') as f:
            data = f.read()
            numbers = []
            # Читаем по 4 байта (размер одного числа)
            for i in range(0, len(data), 4):
                chunk = data[i:i+4]
                if len(chunk) < 4:
                    print(f"⚠️ Неполные данные в {filename}: остаток {len(chunk)} байт")
                    break
                # Распаковываем число
                num = struct.unpack("<I", chunk)[0]  # <I = little-endian unsigned int
                numbers.append(num)
            return numbers
    except Exception as e:
        print(f"🚨 Ошибка при чтении {filename}: {e}")
        return []

# Пример использования:
if __name__ == "__main__":
    for filepath in glob.glob("/mnt/d/vscode_projects/eeve/notebooks/minhash_parallel/remove_ids/*.remove"):
        numbers = read_remove_file(filepath)
        print(f"📁 {filepath} содержит числа: {numbers}")

In [ ]:
PUNCTUATION = "!/—”:％１〈&(、━\\【#%「」，】；+^]~“《„';’{|∶´[=-`*．（–？！：$～«〉,><》)?）。…@_.\"}►»" + "".join(
    map(
        chr,
        (x for a, b in ((0, 9), (11, 13), (13, 32), (127, 160)) for x in range(a, b)),
    )
)

In [ ]:
# SONNET
from datatrove.executor.local import LocalPipelineExecutor
from datatrove.pipeline.dedup import MinhashDedupSignature
from datatrove.pipeline.dedup.minhash import (
    MinhashConfig,
    MinhashDedupBuckets,
    MinhashDedupCluster,
    MinhashDedupFilter,
    MinhashBuildIndex
)
from datatrove.pipeline.readers import JsonlReader
from datatrove.pipeline.writers.jsonl import JsonlWriter
from datatrove.utils.hashing import HashConfig
from datatrove.utils.typeshelper import Languages
from pathlib import Path

# Настройки конфигурации MinHash
minhash_config = MinhashConfig(
    hash_config=HashConfig(precision=64),
    num_buckets=14,
    hashes_per_bucket=8,
)

# Пути к данным
BASE_PATH = Path("./minhash_dedup")
LOGS_FOLDER = BASE_PATH / "logs"
FIRST_DATASET_PATH = "path/to/first/dataset"  # путь к первому (эталонному) датасету
SECOND_DATASET_PATH = "path/to/second/dataset"  # путь ко второму датасету
TOTAL_TASKS = 100  # количество задач, можно настроить в зависимости от размера датасета

# Создаем необходимые директории
BASE_PATH.mkdir(exist_ok=True)
LOGS_FOLDER.mkdir(exist_ok=True)

# Этап 1: Вычисление сигнатур для первого датасета
stage1_first = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(FIRST_DATASET_PATH),
        MinhashDedupSignature(
            output_folder=BASE_PATH / "signatures_first",
            config=minhash_config,
            language=Languages.english
        ),
    ],
    tasks=TOTAL_TASKS,
    workers=8,  # количество параллельных задач
    logging_dir=LOGS_FOLDER / "signatures_first",
)

# Этап 2: Создание индекса из сигнатур первого датасета
stage2_index = LocalPipelineExecutor(
    pipeline=[
        MinhashBuildIndex(
            input_folder=BASE_PATH / "signatures_first",
            output_folder=BASE_PATH / "index_first",
            index_name="first_dataset",
            config=minhash_config,
        ),
    ],
    tasks=minhash_config.num_buckets,  # по одной задаче на каждый бакет
    workers=minhash_config.num_buckets,
    logging_dir=LOGS_FOLDER / "index_first",
    depends=stage1_first,
)

# Этап 3: Вычисление сигнатур для второго датасета
stage3_second = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(SECOND_DATASET_PATH),
        MinhashDedupSignature(
            output_folder=BASE_PATH / "signatures_second",
            config=minhash_config,
            language=Languages.english
        ),
    ],
    tasks=TOTAL_TASKS,
    workers=8,
    logging_dir=LOGS_FOLDER / "signatures_second",
    depends=stage2_index,
)

# Этап 4: Нахождение дубликатов между вторым датасетом и индексом первого
# Используем only_dedup_in_index=True чтобы сравнивать только с индексом
stage4_buckets = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupBuckets(
            input_folder=BASE_PATH / "signatures_second",
            output_folder=BASE_PATH / "buckets_second",
            index_folder=BASE_PATH / "index_first",  # используем индекс первого датасета
            config=minhash_config,
            only_dedup_in_index=True,  # только дедупликация относительно индекса
        ),
    ],
    tasks=minhash_config.num_buckets,
    workers=minhash_config.num_buckets,
    logging_dir=LOGS_FOLDER / "buckets_second",
    depends=stage3_second,
)

# Этап 5: Кластеризация дубликатов
stage5_cluster = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupCluster(
            input_folder=BASE_PATH / "buckets_second",
            output_folder=BASE_PATH / "remove_ids_second",
            config=minhash_config,
            ignore_index_matches=False,  # важно: мы хотим учитывать совпадения с индексом
        ),
    ],
    tasks=1,  # кластеризация выполняется в одной задаче
    workers=1,
    logging_dir=LOGS_FOLDER / "clusters_second",
    depends=stage4_buckets,
)

# Этап 6: Фильтрация второго датасета (удаление дубликатов)
stage6_filter = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(SECOND_DATASET_PATH),
        MinhashDedupFilter(
            input_folder=BASE_PATH / "remove_ids_second",
            exclusion_writer=JsonlWriter(BASE_PATH / "removed_from_second"),  # опционально: сохраняем удаленные документы
        ),
        JsonlWriter(output_folder=BASE_PATH / "deduplicated_second"),  # сохраняем дедуплицированный второй датасет
    ],
    tasks=TOTAL_TASKS,
    workers=8,
    logging_dir=LOGS_FOLDER / "filter_second",
    depends=stage5_cluster,
)

# Запуск пайплайна
if __name__ == "__main__":
    # Запуск последнего этапа автоматически запустит все предыдущие
    stage6_filter.run()

In [ ]:
# DEEPSEEK
from datatrove.executor.local import LocalPipelineExecutor
from datatrove.pipeline.dedup.minhash import (
    MinhashConfig,
    MinhashDedupSignature,
    MinhashDedupBuckets,
    MinhashDedupCluster,
    MinhashDedupFilter,
)
from datatrove.pipeline.readers import JsonlReader
from datatrove.pipeline.writers.jsonl import JsonlWriter
from datatrove.utils.hashing import HashConfig


# Конфигурация MinHash (можно настроить под свои нужды)
minhash_config = MinhashConfig(
    hash_config=HashConfig(precision=64),
    num_buckets=14,
    hashes_per_bucket=8,
)

# Пути к данным
DATASET1_PATH = "path/to/dataset1"  # Первый датасет (база для дедупликации)
DATASET2_PATH = "path/to/dataset2"  # Второй датасет (из него удаляем дубли)
OUTPUT_BASE = "path/to/output"      # Папка для результатов

# Этап 1: Генерация сигнатур для обоих датасетов
stage1_dataset1 = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(DATASET1_PATH),
        MinhashDedupSignature(
            output_folder=f"{OUTPUT_BASE}/signatures_dataset1",
            config=minhash_config,
        ),
    ],
    tasks=100,  # Количество задач (файлов) в датасете
)

stage1_dataset2 = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(DATASET2_PATH),
        MinhashDedupSignature(
            output_folder=f"{OUTPUT_BASE}/signatures_dataset2",
            config=minhash_config,
        ),
    ],
    tasks=100,
    depends=stage1_dataset1,  # Запустится после завершения stage1_dataset1
)

# Этап 2: Поиск дубликатов между датасетами
stage2 = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupBuckets(
            input_folder=f"{OUTPUT_BASE}/signatures_dataset2",  # Сигнатуры второго датасета
            output_folder=f"{OUTPUT_BASE}/buckets",
            index_folder=f"{OUTPUT_BASE}/signatures_dataset1",  # Сигнатуры первого датасета (индекс)
            config=minhash_config,
            only_dedup_in_index=True,  # Игнорируем дубли внутри dataset2
        ),
    ],
    tasks=minhash_config.num_buckets,  # Одна задача на ведро
    depends=stage1_dataset2,
)

# Этап 3: Кластеризация дубликатов
stage3 = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupCluster(
            input_folder=f"{OUTPUT_BASE}/buckets",
            output_folder=f"{OUTPUT_BASE}/remove_ids",
            config=minhash_config,
        ),
    ],
    tasks=1,
    depends=stage2,
)

# Этап 4: Фильтрация дубликатов
stage4 = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(DATASET2_PATH),  # Читаем исходные данные (dataset2)
        MinhashDedupFilter(
            input_folder=f"{OUTPUT_BASE}/remove_ids",
            exclusion_writer=JsonlWriter(f"{OUTPUT_BASE}/removed"),  # Сохраняем удаленные дубли
        ),
        JsonlWriter(output_folder=f"{OUTPUT_BASE}/deduplicated_output"),  # Результат
    ],
    tasks=100,  # Должно совпадать с stage1_dataset2.tasks
    depends=stage3,
)

# Запуск всего пайплайна
stage4.run()